# Συμπερασματολογία και Επικύρωση (Inference and Validation)

Τώρα που έχετε ένα εκπαιδευμένο δίκτυο, μπορείτε να το χρησιμοποιήσετε για να κάνετε προβλέψεις. Η διαδικασία αυτή συνήθως λέγετε **συμπερασματολογία**/**inference**, ένας όρος που δανειστήκαμε από τη στατιστική. Παρόλα αυτά, τα νευρωνικά δίκτυα έχουν την τάση να λειτουργούν *υπερβολικά καλά* στα δεδομένα εκπαίδευσης αλλά δεν μπορούν να γενικεύσουν σε δεδομένα που δεν έχουν ξαναδεί. Το πρόβλημα αυτό ονομάζεται **υπερπροσαρμογή**/**overfitting** και μειώνει την απόδοση των συμπερασμάτων του δικτύου. Για να ελέγξω για υπερπροσαρμογή κατά την εκπαίδευση, μετρούμε την απόδοση σε δεδομένα που δεν εμπεριέχονται στα δεδομένα εκπαίδευσης τα οποία ονομάζονται δεδομένα επαλήθευσης/επικύρωσης **validation**. Μειώνουμε την υπερπροσαρμογή μέσω κανονικοποίησης (regularization) όπως με χρήση dropout (Περιορισμός Ενεργοποίησης Κόμβων) ενώ while παρακολουθώ την απόδοση επικύρωσης (validation performance) κατά τη διάρκεια της εκπαίδευσης. Σε αυτό το notebook, θα δούμε πως μπορούμε να το επιτύχουμε αυτό με τη χρήση της PyTorch.

Όπως συνήθως, ξεκινάμε με το να φορτώσουμε τα δεδομένα μας μέσω της torchvision. Θα δούμε περισσότερες λεπτομέρειες για τη torchvision και πως φορτώνουμε τα δεδομένα μας σε επόμενο στάδιο. Σε αυτό το παράδειγμα θα εκμεταλλευτώ τα δεδομένα ελέγχου τα οποία μπορούμε να πάρουμε με την επιλογή `train=False` όπως παρακάτω:

```python
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
```

Το σετ των δεδομένων ελέγχου περιλαμβάνει εικόνες όπως του σετ εκπαίδευσης. Συνήθως θα δείτε οτι ένα 10-20% του αρχικού dataset δεσμεύεται για να αποτελέσει τα σετ των δεδομένων ελέγχου και επικύρωσης, ενώ το υπόλοιπο ποσοστό αποτελεί τα δεδομένα εκπαίδευσης.

In [ ]:
import torch
from torchvision import datasets, transforms

# Καθορίστε ένα μετασχηματισμό για τη κανονικοποίηση των δεδομένων
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Κατεβάστε και φορτώστε τα δεδομένα εκπαίδευσης
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Κατεβάστε και φορτώστε τα δεδομένα ελέγχου
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

Εδώ θα δημιουργήσω ένα μοντέλο όπως συνήθως, χρησιμοποιώντας το ίδιο μοντέλο με το προηγούμενο notebook.

In [ ]:
from torch import nn, optim
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        # μετατρέπω τον τανυστή εισόδου σε διανυσμα γραμμής. Το x.shape[0] είναι ο αριθμός παρτίδας.
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

Ο στόχος της επικυρωσης ειναι να μετρήσει την αποδοση του δικτύου σε δεδομένα τα οποία δεν εμπεριέχονται στο σετ δεδομένων ελεγχου. Η μετρική της απόδοσης σε αυτή τη περίπτωση ορίζεται απο τον προγραμματιστή. Συνήθως ορίζεται ως ακρίβεια, δηλαδή το ποσοστό των κλάσεων που το δίκτυό μας προβλέπει σωστά. Άλλες μετρικές είναι η Ακρίβεια και Ανάκλησης [precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall#Definition_(classification_context)) και η αναλογία σφάλματος στο πρώτα 5 αποτελέσματα (top-5 error rate). Ε΄δω θα επικρεντρωθουμε στην ακρίβεια (accuracy). Πρώτα κάνω ένα πέρασμα προς το εμπρός με μία παρτίδα απο το σετ ελέγχου.

In [ ]:
model = Classifier()

images, labels = next(iter(testloader))
# Υπολόγισε τις πιθανότητες καθε κλάσης
ps = torch.exp(model(images))
# Κάνω έναν έλεγχο εαν έχει σωστή μορφή ο τανυστής, πρέπει να έχουμε πιθανότητες και για τις 10 κλάσεις για τις 64 εικόνες
print(ps.shape)

Μέσω των πιθανοτήτων, μπορώ να πάρω τη πιο πιθανή κλάση χρησιμοποιώντας τη μέθοδο `ps.topk`. Αυτή επιστρέφει τις $k$ υψηλότερες τιμές. Καθώς θέλω να πάρω την πιο πιθανή κλάση, χρησιμοποιώ `ps.topk(1)`, η οποία επιστρέφει τις top-$k$ τιμές και τους top-$k$ δείκτες. Αν η υψηλότερη τιμή είναι στο πέμπτο στοιχείο, θα λάβουμε τον δείκτη 4.

In [ ]:
top_p, top_class = ps.topk(1, dim=1)
# Ψάξε για τις πιο πιθανές κλάσεις στα 10 πρώτα δείγματα
print(top_class[:10,:])

Τώρα μπορώ να ελέγξω εάν οι κλάσεις που έχουν προβλεφθεί είναι ίδιες με τις ετικέτες των εικόνων αυτών. Αυτό μπορώ να το πετύχω εύκολα με την ισότητα μεταξύ `top_class` και `labels`, αλλα πρέπει να ήμαστε προσεκτικοί με τις διαστάσεις. Εδώ η `top_class` είναι ένας τανυστής 2D με μέγεθος `(64, 1)` ενώ η `labels` είναι 1D με μέγεθος `(64)`. Για να δουλέψει σωστά ο έλεγχος ισότητας όπως θέλουμε, πρέπει οι `top_class` και `labels` να έχουν το ίδιο μέγεθος.

Αν εκτελέσω

```python
equals = top_class == labels
```

η `equals` θα έχει μέγεθος `(64, 64)`, δοκιμάστε το. Αυτό που κάνει είναι να συγκρίνει το πρώτο στοιχείο σε κάθε γραμμή της `top_class` μ κάθε στοιχείο της `labels` και επιστρέφει 64 True/False boolean τιμές για κάθε γραμμή.

In [ ]:
equals = top_class == labels.view(*top_class.shape)

Τώρα πρέπει να υπολογίσω το ποσοστό των σωστών προβλέψεων. Η `equals` έχει binary τιμές, είτε 0 είτε 1. Αυτο σημάινει οτι αν απλα αθροίσουμε όλες τις τιμές και τις διαιρέσουμε με τον αριθμό των τιμών, θα πάρουμε το ποσοστό των σωστών προβλέψεων. Αυτό ειναι η ίδια πράξη με το να παίρναμε τη μέση τιμή, οπότε μπορούμε να πάρουμε την ακρίβεια καλώντας την `torch.mean`. Μακάρι όμως να ήταν τοσο απλό. Αν το δοκιμάσετε με `torch.mean(equals)`, θα λάβεται ένα σφάλμα

```
RuntimeError: mean is not implemented for type torch.ByteTensor
```

Αυτό συμβαίνει γιατι η `equals` έχει τύπο `torch.ByteTensor` αλλά η `torch.mean` δεν υποστηρίζει τανυστές αυτού του είδους. Οπότε πρέπει να μετατρέψουμε την `equals` σε τανυστή float. Προσέξτε οτι οταν εκτελώ την `torch.mean` επιστρέφει ενα τανυστή scalar, για να λάβω την πραγματική τιμή ως float πρέπει να εκτελέσουμε `accuracy.item()`.

In [ ]:
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')

Το δίκτυο δεν έχει εκπαιδευτεί ακόμα οπότε κάνει τυχαίες προβλέψεις με αποτέλεσμα όντως να βλέπουμε μία ακρίβεια περίπου στο 10%. Τώρα ας εκπαιδεύσουμε το δίκτυό μας και να συμπεριλάβουμε ένα πέρασμα επικύρωσης έτσι ώστε να μπορέσουμε να υπολογίσουμε πόσο καλά αποδίδει το δίκτυό μας στο σετ ελέγχου. Από τη στιγμή που δεν θα κάνω update των παραμέτρων στο βήμα επικύρωσης, μπορούμε να επιταχύνουμε τη διαδικασία απενεργοποιώντας τις παραγώγους με την `torch.no_grad()`:

```python
# απενεργοποιώ τις παραγώγους
with torch.no_grad():
    # βήμα επικύρωσης
    for images, labels in testloader:
        ...
```

>**Άσκηση:** Εφαρμόστε τον βρόχο επικύρωσης παρακάτω. Μπορείτε σε μεγάλο βαθμό να αντιγράψετε και να επικολλήσετε τον κώδικα από πάνω, αλλά προτείνω να τον πληκτρολογήσετε εσείς καθώς είναι απαραίτητο να αναπτύξετε τις δεξιότητές σας. Σε γενικές γραμμές, θα μαθαίνετε πάντα περισσότερα με το να γράφετε τον κώδικα και όχι να τον αντιγράφετε.

In [ ]:
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 30
steps = 0

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0
        
        # Απενεργοποιώ τις παραγωγους στο βρόχο επικύρωσης καθώς μου απελευθερώνει μνήμη και απαιτεί λιγότερες πράξεις
        with torch.no_grad():
            for images, labels in testloader:
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Εποχή: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)

## Υπερπροσαρμογή (Overfitting)

Αν παρατηρήσουμε τις απώλειες (σφαλμα) της εκπαίδευσης και της επικύρωσης καθώς εκπαιδεύουμε το δίκτυο, θα παρατηρήσουμε ένα φαινομενο το οποίο είναι γνωστό ως υπερπροσαρμογή (overfitting).

<img src='assets/overfitting.png' width=450px>

Το δίκτυο μαθαίνει το σετ εκπαίδευσης όλο και καλύτερα, με αποτέλεσμα χαμηλές απώλειες εκπαίδευσης. Παρόλα αυτά, αρχίζει και έχει προβλήματα γενίκευσης σε δεδομένα εκτός του σετ εκπαίδευσης με αποτέλεσμα οι απώλειες επικύρωσης να αυξάνουν. Ο απώτερος στόχος κάθε νευρωνικού δικτύου είναι να κάνει σωστές προβλέψεις σε νέα δεδομένα, οπότε θα πρέπει πάση θυσία να έχουμε οσο το δυνατό χαμηλότερο σφάλμα επικύρωσης. Μία λύση είναι να χρησιμοποιήσουμε την έκδοση του δικτύου εκείνη στις οποίες οι παράμετροί του δίνουν το χαμηλότερο σφάλμα επικύρωσης, στο συγκεκριμένο παράδειγμα μεταξύ της εποχής 8-10. Αυτή η στρατηγική ονομάζεται *πρόωρη διακοπή*/*early-stopping*. Στη πράξη, σώζουμε το δίκτυό μας συχνά καθώς το εκπαιδεύουμε και αργότερα επιλέγουμε το μοντέλο με το χαμηλότερο σφάλμα επικύρωσης.

Η πιο διαδεδομένη τεχνική για να αποφεύγουμε το φαινόμενο της υπερπροσαρμογής (πέραν της πρόωρης διακοπής) είναι το dropout (Περιορισμός Ενεργοποίησης κόμβων), όπου τυχαία απενεργοποιούμε κόμβους εισόδου. Αυτή η τεχνική αναγκάζει το δίκτυο να μοιράζεται τη πληροφορία σε όλα τα βάρη, αυξάνοντας την ικανότητα γενίκευσης σε νέα δεδομένα. Η εφαρμογή dropout στη PyTorch είναι πολύ απλή χρησιμοποιώντας την μέθοδο nn.Dropout. [`nn.Dropout`](https://pytorch.org/docs/stable/nn.html#torch.nn.Dropout).

```python
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
        # Dropout με 0.2 πιθανότητα απενεργοποίησης κόμβων 
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        #  μετατρέπω τον τανυστή εισόδου σε διανυσμα γραμμής
        x = x.view(x.shape[0], -1)
        
        # Εφαρμογή dropout
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        
        # έξοδος, οπότε δεν χρησιμοποιώ εδώ dropout
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x
```

Κατά την εκπαίδευση θέλουμε να χρησιμοποιήσουμε dropout για την αποφυγή υπερπροσαρμογής, αλλά κατά τη διαδικασία συμπερασματολογίας θέλουμε να χρησιμοποιήσουμε όλο το δίκτυο. Οπότε πρέπει να απενεργοποιήσω το dropout κατά την επικύρωση και τον έλεγχο, και φυσικά όποτε χρησιμοποιώ το δίκτυο για προβλέψεις. Για να το πετύχω αυτό, χρησιμοποιώ την `model.eval()`. Η εντολή αυτή προσαρμόζει το μοντέλο μας σε κατάσταση αξιολόγησης όπου η πιθανότητα dropout είναι 0. Μπορείτε να επαναφέρετε το dropout πίσω ορίζοντας το μοντέλο σας σε κατάσταση εκπαίδευσης με την `model.train()`. Γενικά, το πλάνο για το βρόχο επικύρωσης θα είναι όπως παρακάτω, όπου απενεργοποιείτε τις παραγώγους, βάζετε το μοντέλο σας σε κατάσταση αξιολόγησης, υπολογίζετε το σφάλμα επικύρωσης και τη μετρική, και επιστρέφετε το μοντέλο σας πίσω σε κατάσταση εκπαίδευσης.

```python
# απενεργοποίηση παραγώγων
with torch.no_grad():
    
    # ορίστε το μοντέλο σας σε κατασταση επικύρωσης
    model.eval()
    
    # βρόχος επίκύρωσης
    for images, labels in testloader:
        ...

# επαναφέρετε ξανα το μοντέλο σε κατασταση εκπαίδευσης
model.train()
```

> **Άσκηση:** Εφαρμόστε τη στρατηγική dropout στο μοντέλο σας και εκπαιδεύστε το ξανά στο Fashion-MNIST. Δείτε εάν επιτυγχάνεται χαμηλότερο σφάλμα επικύρωσης.

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)

        # μονάδα Dropout (Περιορισμός Ενεργοποίησης) με 0.2 πιθανότητα απόρριψης
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        # βεβαιωθείτε οτι ο τανυστής εισόδου έχει "ισοπεδωθεί" (flattened) ή αλλιώς έχει γίνει γραμμή
        x = x.view(x.shape[0], -1)

        # Χρήση dropout
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))

        # έξοδος, οπότε δεν χρησιμοποιώ εδώ dropout
        x = F.log_softmax(self.fc4(x), dim=1)

        return x

In [ ]:
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 30
steps = 0

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0
        
        # Απενεργοποιώ τις παραγώγους για την επικύρωση για λιγότερη μνήμη και υπολογισμούς
        with torch.no_grad():
            model.eval()
            for images, labels in testloader:
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        model.train()
        
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Εποχή: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(train_losses[-1]),
              "Test Loss: {:.3f}.. ".format(test_losses[-1]),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)

## Συμπερασματολογία/Inference

Τώρα που το μοντέλο είναι εκπαιδευμένο, μπορούμε να το χρησιμοποιήσουμε για προβλέψεις. Το έχουμε ξανακάνει, αλλά τώρα πρέπει να θυμηθούμε να βάλουμε το μοντέλο μας σε κατάσταση συμπερασματολογίας με την εντολή `model.eval()`. Επίσης πρέπει να απενεργοποιήσουμε το autograd με την `torch.no_grad()`.

In [ ]:
# Εισαγωγή του helper module (υπάρχει στον υποφάκελο)
import helper

# Ελέξτε το δικτυό σας!

model.eval()

dataiter = iter(testloader)
images, labels = dataiter.next()
img = images[0]
# Μετετρεψε την 2D εικόνα σε 1D διάνυσμα
img = img.view(1, 784)

# Υπολόγισε τις πιθανότητες κλάσης (softmax) για την img
with torch.no_grad():
    output = model.forward(img)

ps = torch.exp(output)

# Εμφάνισε την εικόνα και τις πιθανότητες
helper.view_classify(img.view(1, 28, 28), ps, version='Fashion')

## Στη συνέχεια!

Στο επόμενο μέρος, θα δούμε πώς μπορούμε να αποθηκεύσουμε τα εκπαιδευμένα μας δίκτυα. Στη πραγματικότητα, δεν θέλουμε να εκπαιδεύουμε το δίκτυό μας κάθε φορά που θέλουμε να το χρησιμοποιήσουμε. Αντί αυτού, το εκπαιδεύουμε μόνο μία φορά, το αποθηκεύουμε, και στη συνέχεια το φορτώνουμε όταν θέλουμε να το εκπαιδεύσουμε περισσότερο ή να το χρησιμοποιήσουμε για συμπερασματολογία.

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

Αυτό το notebook 📖 δημιουργήθηκε για το μάθημα ***Υπολογιστική Νοημοσύνη και Μηχανική Μάθηση*** του Τμήματος Μηχανικών Παραγωγής και Διοίκησης, της Πολυτεχνικής Σχολής του Δημοκριτείου Πανεπιστημίου Θράκης.<br>
This notebook is made available under the Creative Commons Attribution [(CC-BY)](https://creativecommons.org/licenses/by/4.0/legalcode) license. Code is also made available under the [MIT License](https://opensource.org/licenses/MIT).<br>
Author: Asst. Prof. Angelos Amanatiadis
<img src="assets/cc.png" style="width:55px; float: right; margin: 0px 0px 0px 0px;"></img>
<img src="assets/mit.png" style="width:40px; float: right; margin: 0px 10px 0px 0px;"></img>